In [1]:
import ast
import json
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import StrMethodFormatter
from matplotlib.dates import DateFormatter
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
from sklearn import metrics 
import defi.defi_tools as dft


pools_daily = pd.read_csv('pools_daily_weth_btc.csv', delimiter=';', encoding='utf-8')
pools_daily

,ID,Date,Date (Unix),Pool,Pool Tokens,Liquidity,Sqrt Price,Token0 Price,Token1 Price,Tick,...,Open (WBTC),High (WBTC),Low (WBTC),Close (WBTC),ETH Price CoinGecko,ETH Market Cap CoinGecko,ETH Volume CoinGecko,BTC Price CoinGecko,BTC Market Cap CoinGecko,BTC Volume CoinGecko
0,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed-18779,2021-01-06,1622505600,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,1.177975e+18,2.962947e+34,0.071501,13.985840,"256,651",...,0.072025,0.072271,0.070739,0.072025,1103.358252,1.251129e+11,4.714825e+10,34082.206049,6.336517e+11,6.742050e+10
1,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed-18809,2021-01-07,1625097600,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,5.294422e+17,3.152406e+34,0.063165,15.831613,"257,891",...,0.064758,0.064758,0.062929,0.064758,1208.575093,1.373068e+11,4.788685e+10,36933.520137,6.827437e+11,7.102720e+10
2,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed-18840,2021-01-08,1627776000,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,8.107288e+17,3.127074e+34,0.064192,15.578191,"257,730",...,0.060816,0.064744,0.060816,0.060816,1229.471315,1.390849e+11,4.304288e+10,39547.084081,7.353397e+11,7.864296e+10
3,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed-18871,2021-01-09,1630454400,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,4.912223e+17,2.837729e+34,0.077950,12.828698,"255,788",...,0.072510,0.077950,0.072510,0.072510,1223.729688,1.391457e+11,4.813262e+10,40815.961855,7.589710e+11,8.284647e+10
4,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed-18901,2021-01-10,1633046400,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,7.104710e+17,3.025238e+34,0.068587,14.580081,"257,067",...,0.068587,0.069414,0.067953,0.068587,1282.979576,1.473979e+11,3.485112e+10,40296.529004,7.493472e+11,5.592181e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed-19711,2023-12-20,1703030400,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,5.725465e+17,3.530877e+34,0.050349,19.861223,"260,159",...,0.051558,0.051754,0.048452,0.051558,2176.839413,2.616225e+11,1.525800e+10,42250.206183,8.274833e+11,2.340252e+10
961,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed-19712,2023-12-21,1703116800,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,6.084813e+17,3.506200e+34,0.051061,19.584572,"260,018",...,0.050349,0.051509,0.050349,0.050349,2198.421404,2.641199e+11,1.915574e+10,43634.124168,8.541739e+11,2.833465e+10
962,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed-19713,2023-12-22,1703203200,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,7.203820e+17,3.444140e+34,0.052917,18.897420,"259,661",...,0.051061,0.053450,0.051035,0.051061,2236.186962,2.686619e+11,1.946674e+10,43849.699599,8.583121e+11,2.194864e+10
963,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed-19714,2023-12-23,1703289600,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,8.436157e+17,3.450490e+34,0.052723,18.967168,"259,698",...,0.052917,0.052917,0.052372,0.052917,2327.048963,2.798706e+11,2.378392e+10,44003.696022,8.618227e+11,2.083394e+10


In [31]:
swaps = pd.read_csv('swaps.csv', delimiter=';', encoding='utf-8')
swaps = swaps.sort_values(by=['Timestamp (Unix)']).reset_index(drop=True)
swaps['Tick'] = swaps['Tick'].apply(lambda x: int(x.replace(',', '')))
swaps['Current Day'] = pd.to_datetime(swaps['Timestamp'], format="%d/%m/%Y, %H:%M:%S")
swaps['Current Day'] = swaps['Current Day'].dt.date
swaps['Current Hour'] = pd.to_datetime(swaps['Timestamp'], format="%d/%m/%Y, %H:%M:%S")
swaps['Current Hour'] = swaps['Current Hour'].apply(lambda x: x.replace(minute=0, second=0))

swaps.to_csv('swaps_cleaned.csv', sep=';', index=False)
swaps


,ID,Transaction ID,Timestamp,Timestamp (Unix),Pool,Sender,Recipient,Origin,Token 0 Symbol,Token 1 Symbol,Token 0,Token 1,Amount 0,Amount 1,Amount USD,SqrticeX96,Tick,Log Index,Current Day,Current Hour
0,0x9136a3addf5ea98730b4cd1832c25a12001162cbda7c...,0x9136a3addf5ea98730b4cd1832c25a12001162cbda7c...,"05/05/2021, 04:22:20",1620159740,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0xdd0d6c26a03d6f6541471d44179f56d478f50f6b,0xdd0d6c26a03d6f6541471d44179f56d478f50f6b,WBTC,WETH,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,-0.000622,0.010000,0.000000,NaN,258019,134,2021-05-05,2021-05-05 04:00:00
1,0x472f6c3a372cb7e823ed82ef5a892a207af401afdcd2...,0x472f6c3a372cb7e823ed82ef5a892a207af401afdcd2...,"05/05/2021, 04:44:52",1620161092,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0x11921c9c14ba2ccd34cef17c01c0ef36ffad8713,0x11921c9c14ba2ccd34cef17c01c0ef36ffad8713,WBTC,WETH,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,-0.000621,0.010000,0.000000,NaN,258033,295,2021-05-05,2021-05-05 04:00:00
2,0x750b4d06db061fdc59cdf21a3f5a24518eb0f469a255...,0x750b4d06db061fdc59cdf21a3f5a24518eb0f469a255...,"05/05/2021, 05:22:07",1620163327,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0x39336a270b92a64a68925fb8540e7b4adf218e0a,0x39336a270b92a64a68925fb8540e7b4adf218e0a,WBTC,WETH,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,-0.000620,0.010000,0.000000,NaN,258048,349,2021-05-05,2021-05-05 05:00:00
3,0x0b4a29f4ce37f0c53300209985a4ac78e6d8f2b38b12...,0x0b4a29f4ce37f0c53300209985a4ac78e6d8f2b38b12...,"06/05/2021, 02:16:19",1620238579,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0xe59a4be85f063f1080a0443e6a5fa1dc398a9421,0xe59a4be85f063f1080a0443e6a5fa1dc398a9421,WBTC,WETH,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,-0.061207,1.000000,1714.019835,NaN,258305,223,2021-05-06,2021-05-06 02:00:00
4,0x73019fd9817196abc3006265899d970ab18e3078f9d0...,0x73019fd9817196abc3006265899d970ab18e3078f9d0...,"06/05/2021, 02:46:25",1620240385,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0x05ab4c6ab83658e88ff785ca1e78d1519762badb,0x05ab4c6ab83658e88ff785ca1e78d1519762badb,WBTC,WETH,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,-0.030017,0.500000,851.814472,NaN,258434,151,2021-05-06,2021-05-06 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200142,0x9af0035b5112f5e8d50d67483c80fb2224c7930fcd3b...,0x9af0035b5112f5e8d50d67483c80fb2224c7930fcd3b...,"03/01/2024, 09:41:59",1704246119,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,0x00000000d7c185343e6504e428b8f8b5ad6c91b8,0x00000000d7c185343e6504e428b8f8b5ad6c91b8,WBTC,WETH,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,-0.068260,1.305000,3083.413694,NaN,259747,500,2024-01-03,2024-01-03 09:00:00
200143,0x8027472a11589e1b350cbe2dd8b5be17777ab1e28eff...,0x8027472a11589e1b350cbe2dd8b5be17777ab1e28eff...,"03/01/2024, 11:14:11",1704251651,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0xb64619712b73fb0e6740040af205a0f8134ef8f9,WBTC,WETH,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,-1.121753,21.447268,50815.266478,NaN,259749,29,2024-01-03,2024-01-03 11:00:00
200144,0xd8aa26bf8fc3674f9acec6c88fbe3c74583a35e191a4...,0xd8aa26bf8fc3674f9acec6c88fbe3c74583a35e191a4...,"03/01/2024, 11:14:23",1704251663,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0xf72837a9fce033d57094f52e4f88f129502fa067,WBTC,WETH,0x2260fac5e5542a773aa44f

In [39]:
poolHour = pd.read_csv('poolHourData_cleaned.csv', delimiter=';', encoding='utf-8')
poolHour = poolHour.sort_values(by=['Period Start']).reset_index(drop=True)
poolHour['Period Start'] = pd.to_datetime(poolHour['Period Start'], format="%Y-%m-%d %H:%M:%S")

In [40]:
# Merge with pool hour to get out the current tick
swaps = pd.merge(swaps, poolHour[['Period Start', 'Tick']], left_on='Current Hour', right_on='Period Start', how='left')


,ID,Transaction ID,Timestamp,Timestamp (Unix),Pool,Sender,Recipient,Origin,Token 0 Symbol,Token 1 Symbol,...,Amount 0,Amount 1,Amount USD,SqrticeX96,Tick_x,Log Index,Current Day,Current Hour,Period Start,Tick_y
0,0x9136a3addf5ea98730b4cd1832c25a12001162cbda7c...,0x9136a3addf5ea98730b4cd1832c25a12001162cbda7c...,"05/05/2021, 04:22:20",1620159740,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0xdd0d6c26a03d6f6541471d44179f56d478f50f6b,0xdd0d6c26a03d6f6541471d44179f56d478f50f6b,WBTC,WETH,...,-0.000622,0.010000,0.000000,NaN,258019,134,2021-05-05,2021-05-05 04:00:00,2021-05-05 04:00:00,258033
1,0x472f6c3a372cb7e823ed82ef5a892a207af401afdcd2...,0x472f6c3a372cb7e823ed82ef5a892a207af401afdcd2...,"05/05/2021, 04:44:52",1620161092,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0x11921c9c14ba2ccd34cef17c01c0ef36ffad8713,0x11921c9c14ba2ccd34cef17c01c0ef36ffad8713,WBTC,WETH,...,-0.000621,0.010000,0.000000,NaN,258033,295,2021-05-05,2021-05-05 04:00:00,2021-05-05 04:00:00,258033
2,0x750b4d06db061fdc59cdf21a3f5a24518eb0f469a255...,0x750b4d06db061fdc59cdf21a3f5a24518eb0f469a255...,"05/05/2021, 05:22:07",1620163327,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0x39336a270b92a64a68925fb8540e7b4adf218e0a,0x39336a270b92a64a68925fb8540e7b4adf218e0a,WBTC,WETH,...,-0.000620,0.010000,0.000000,NaN,258048,349,2021-05-05,2021-05-05 05:00:00,2021-05-05 05:00:00,258048
3,0x0b4a29f4ce37f0c53300209985a4ac78e6d8f2b38b12...,0x0b4a29f4ce37f0c53300209985a4ac78e6d8f2b38b12...,"06/05/2021, 02:16:19",1620238579,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0xe59a4be85f063f1080a0443e6a5fa1dc398a9421,0xe59a4be85f063f1080a0443e6a5fa1dc398a9421,WBTC,WETH,...,-0.061207,1.000000,1714.019835,NaN,258305,223,2021-05-06,2021-05-06 02:00:00,2021-05-06 02:00:00,258434
4,0x73019fd9817196abc3006265899d970ab18e3078f9d0...,0x73019fd9817196abc3006265899d970ab18e3078f9d0...,"06/05/2021, 02:46:25",1620240385,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0x05ab4c6ab83658e88ff785ca1e78d1519762badb,0x05ab4c6ab83658e88ff785ca1e78d1519762badb,WBTC,WETH,...,-0.030017,0.500000,851.814472,NaN,258434,151,2021-05-06,2021-05-06 02:00:00,2021-05-06 02:00:00,258434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200142,0x9af0035b5112f5e8d50d67483c80fb2224c7930fcd3b...,0x9af0035b5112f5e8d50d67483c80fb2224c7930fcd3b...,"03/01/2024, 09:41:59",1704246119,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,0x00000000d7c185343e6504e428b8f8b5ad6c91b8,0x00000000d7c185343e6504e428b8f8b5ad6c91b8,WBTC,WETH,...,-0.068260,1.305000,3083.413694,NaN,259747,500,2024-01-03,2024-01-03 09:00:00,2024-01-03 09:00:00,259747
200143,0x8027472a11589e1b350cbe2dd8b5be17777ab1e28eff...,0x8027472a11589e1b350cbe2dd8b5be17777ab1e28eff...,"03/01/2024, 11:14:11",1704251651,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0xb64619712b73fb0e6740040af205a0f8134ef8f9,WBTC,WETH,...,-1.121753,21.447268,50815.266478,NaN,259749,29,2024-01-03,2024-01-03 11:00:00,2024-01-03 11:00:00,259754
200144,0xd8aa26bf8fc3674f9acec6c88fbe3c74583a35e191a4...,0xd8aa26bf8fc3674f9acec6c88fbe3c74583a35e191a4...,"03/01/2024, 11:14:23",1704251663,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0xf72837a9fce033d57094f52e4f88f129502fa067,WBTC,WETH,...,-0.968853,18.526282,43894.742678,NaN,259750,34,2024-01-03,2024-01-03 11:00:00,2024-01-03 11:00:00,259754
200145,0x532d5c625b6134e52d8358500a062a648dd833c30052...,0x532d5c625b6134e52d8358500a062a648dd833c30052...,"03/01/2024, 11:14:35",1704251675,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x767c8bb1574bee5d4fe35e27e0003c89d43c5121,0x2d722c96f79d149dd21e9ef36f93fc12906ce9f8,0xb2ce0eb5b5134c

In [41]:
swaps.rename(columns={'Tick_x': 'Swap Tick', 'Tick_y': 'Active Tick'}, inplace=True)
swaps = swaps.drop(['Period Start'], axis=1)

In [42]:
ticksdaydata = pd.read_csv('tickdaydata.csv', delimiter=';', encoding='utf-8')
ticksdaydata = ticksdaydata.sort_values(by=['Date']).reset_index(drop=True)
ticksdaydata['Tick Index'] = ticksdaydata['Tick'].apply(lambda x: int(x.split('#')[1]))
ticksdaydata['Current Day'] = pd.to_datetime(ticksdaydata['Date'],format="%d/%m/%Y, %H:%M:%S")
ticksdaydata['Current Day'] = ticksdaydata['Current Day'].dt.date
ticksdaydata = ticksdaydata.sort_values(by=['Current Day']).reset_index(drop=True)
ticksdaydata.to_csv('ticksdaydata_cleaned.csv', sep=';', encoding='utf-8', index="Date")
ticksdaydata


,ID,Date,Date (Unix),Pool,Pool Tokens,Tick,Liquidity Gross,Liquidity Net,Volume Token 0,Volume Token 1,Volume USD,Fees USD,Fee Growth Outside 0 X128,Fee Growth Outside 1 X128,Tick Index,Current Day
0,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#258...,"04/05/2021, 08:00:00",1620086400,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#258600,"246,584,554,095","-246,584,554,095",0.0,0.0,0.0,0.0,0,0,258600,2021-05-04
1,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#255...,"04/05/2021, 08:00:00",1620086400,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#255900,"246,584,554,095","246,584,554,095",0.0,0.0,0.0,0.0,0,"592,453,493,206,141,100,000,000,000,000,000,00...",255900,2021-05-04
2,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#255...,"04/05/2021, 08:00:00",1620086400,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#255540,"11,342,805,944","11,342,805,944",0.0,0.0,0.0,0.0,0,0,255540,2021-05-04
3,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#262...,"04/05/2021, 08:00:00",1620086400,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#262440,"2,154,941,425","-2,154,941,425",0.0,0.0,0.0,0.0,0,0,262440,2021-05-04
4,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#253...,"04/05/2021, 08:00:00",1620086400,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#253320,"34,519,254,610,711","34,519,254,610,711",0.0,0.0,0.0,0.0,0,0,253320,2021-05-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54705,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#260...,"03/01/2024, 08:00:00",1704240000,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#260760,"40,798,821,570,215,864","-40,798,821,570,215,864",0.0,0.0,0.0,0.0,0,0,260760,2024-01-03
54706,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#258...,"03/01/2024, 08:00:00",1704240000,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#258840,"48,240,153,664,540,000","46,060,335,857,442,296",0.0,0.0,0.0,0.0,"32,546,346,412,081,010,000,000,000,000,000","4,721,958,224,723,479,000,000,000,000,000,000,...",258840,2024-01-03
54707,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#251...,"03/01/2024, 08:00:00",1704240000,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#251640,"506,924,731,334,653","506,924,731,334,653",0.0,0.0,0.0,0.0,"9,323,720,797,434,372,000,000,000,000,000","1,417,799,360,942,755,600,000,000,000,000,000,...",251640,2024-01-03
54708,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#264...,"03/01/2024, 08:00:00",1704240000,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC / WETH,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed#264360,"1,672,756,774,698,872","-1,672,756,774,698,872",0.0,0.0,0.0,0.0,0,0,264360,2024-01-03


In [45]:
# For every row in swap, get ticks that are between the current tick and active tick, calculate the sum liquiditynet from ticksdaydata
# and add it to the swap row
def getLiquidityNet(row):
    ticks = ticksdaydata[(ticksdaydata['Tick Index'] >= row['Swap Tick']) & (ticksdaydata['Tick Index'] <= row['Active Tick']) & (ticksdaydata['Current Day'] == row['Current Day'])]
    return ticks['Liquidity Net'].sum()

swaps['LiquidityNet'] = swaps.apply(getLiquidityNet, axis=1)
swaps


,ID,Transaction ID,Timestamp,Timestamp (Unix),Pool,Sender,Recipient,Origin,Token 0 Symbol,Token 1 Symbol,...,Amount 0,Amount 1,Amount USD,SqrticeX96,Swap Tick,Log Index,Current Day,Current Hour,Active Tick,LiquidityNet
0,0x9136a3addf5ea98730b4cd1832c25a12001162cbda7c...,0x9136a3addf5ea98730b4cd1832c25a12001162cbda7c...,"05/05/2021, 04:22:20",1620159740,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0xdd0d6c26a03d6f6541471d44179f56d478f50f6b,0xdd0d6c26a03d6f6541471d44179f56d478f50f6b,WBTC,WETH,...,-0.000622,0.010000,0.000000,NaN,258019,134,2021-05-05,2021-05-05 04:00:00,258033,0
1,0x472f6c3a372cb7e823ed82ef5a892a207af401afdcd2...,0x472f6c3a372cb7e823ed82ef5a892a207af401afdcd2...,"05/05/2021, 04:44:52",1620161092,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0x11921c9c14ba2ccd34cef17c01c0ef36ffad8713,0x11921c9c14ba2ccd34cef17c01c0ef36ffad8713,WBTC,WETH,...,-0.000621,0.010000,0.000000,NaN,258033,295,2021-05-05,2021-05-05 04:00:00,258033,0
2,0x750b4d06db061fdc59cdf21a3f5a24518eb0f469a255...,0x750b4d06db061fdc59cdf21a3f5a24518eb0f469a255...,"05/05/2021, 05:22:07",1620163327,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0x39336a270b92a64a68925fb8540e7b4adf218e0a,0x39336a270b92a64a68925fb8540e7b4adf218e0a,WBTC,WETH,...,-0.000620,0.010000,0.000000,NaN,258048,349,2021-05-05,2021-05-05 05:00:00,258048,0
3,0x0b4a29f4ce37f0c53300209985a4ac78e6d8f2b38b12...,0x0b4a29f4ce37f0c53300209985a4ac78e6d8f2b38b12...,"06/05/2021, 02:16:19",1620238579,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0xe59a4be85f063f1080a0443e6a5fa1dc398a9421,0xe59a4be85f063f1080a0443e6a5fa1dc398a9421,WBTC,WETH,...,-0.061207,1.000000,1714.019835,NaN,258305,223,2021-05-06,2021-05-06 02:00:00,258434,"2,400,506,231,968,713-4,221,963,868,299,692"
4,0x73019fd9817196abc3006265899d970ab18e3078f9d0...,0x73019fd9817196abc3006265899d970ab18e3078f9d0...,"06/05/2021, 02:46:25",1620240385,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0x05ab4c6ab83658e88ff785ca1e78d1519762badb,0x05ab4c6ab83658e88ff785ca1e78d1519762badb,WBTC,WETH,...,-0.030017,0.500000,851.814472,NaN,258434,151,2021-05-06,2021-05-06 02:00:00,258434,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200142,0x9af0035b5112f5e8d50d67483c80fb2224c7930fcd3b...,0x9af0035b5112f5e8d50d67483c80fb2224c7930fcd3b...,"03/01/2024, 09:41:59",1704246119,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,0x00000000d7c185343e6504e428b8f8b5ad6c91b8,0x00000000d7c185343e6504e428b8f8b5ad6c91b8,WBTC,WETH,...,-0.068260,1.305000,3083.413694,NaN,259747,500,2024-01-03,2024-01-03 09:00:00,259747,0
200143,0x8027472a11589e1b350cbe2dd8b5be17777ab1e28eff...,0x8027472a11589e1b350cbe2dd8b5be17777ab1e28eff...,"03/01/2024, 11:14:11",1704251651,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0xb64619712b73fb0e6740040af205a0f8134ef8f9,WBTC,WETH,...,-1.121753,21.447268,50815.266478,NaN,259749,29,2024-01-03,2024-01-03 11:00:00,259754,0
200144,0xd8aa26bf8fc3674f9acec6c88fbe3c74583a35e191a4...,0xd8aa26bf8fc3674f9acec6c88fbe3c74583a35e191a4...,"03/01/2024, 11:14:23",1704251663,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0xf72837a9fce033d57094f52e4f88f129502fa067,WBTC,WETH,...,-0.968853,18.526282,43894.742678,NaN,259750,34,2024-01-03,2024-01-03 11:00:00,259754,0
200145,0x532d5c625b6134e52d8358500a062a648dd833c30052...,0x532d5c625b6134e52d8358500a062a648dd833c30052...,"03/01/2024, 11:14:35",1704251675,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x767c8bb1574bee5d4fe35e27e0003c89d43c5121,0x2d722c96f79d149dd21e9ef36f93fc12906ce9f8,0xb2ce0eb5b5134c734bf1a131257275432f33a4f4,WBTC,WETH,...,-2.052570,39.256197,93007.533053,NaN,259752,37,2024

In [46]:
def getLiquidityGross(row):
    ticks = ticksdaydata[(ticksdaydata['Tick Index'] >= row['Swap Tick']) & (ticksdaydata['Tick Index'] <= row['Active Tick']) & (ticksdaydata['Current Day'] == row['Current Day'])]
    return ticks['Liquidity Gross'].sum()

swaps['LiquidityGross'] = swaps.apply(getLiquidityNet, axis=1)
swaps

,ID,Transaction ID,Timestamp,Timestamp (Unix),Pool,Sender,Recipient,Origin,Token 0 Symbol,Token 1 Symbol,...,Amount 1,Amount USD,SqrticeX96,Swap Tick,Log Index,Current Day,Current Hour,Active Tick,LiquidityNet,LiquidityGross
0,0x9136a3addf5ea98730b4cd1832c25a12001162cbda7c...,0x9136a3addf5ea98730b4cd1832c25a12001162cbda7c...,"05/05/2021, 04:22:20",1620159740,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0xdd0d6c26a03d6f6541471d44179f56d478f50f6b,0xdd0d6c26a03d6f6541471d44179f56d478f50f6b,WBTC,WETH,...,0.010000,0.000000,NaN,258019,134,2021-05-05,2021-05-05 04:00:00,258033,0,0
1,0x472f6c3a372cb7e823ed82ef5a892a207af401afdcd2...,0x472f6c3a372cb7e823ed82ef5a892a207af401afdcd2...,"05/05/2021, 04:44:52",1620161092,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0x11921c9c14ba2ccd34cef17c01c0ef36ffad8713,0x11921c9c14ba2ccd34cef17c01c0ef36ffad8713,WBTC,WETH,...,0.010000,0.000000,NaN,258033,295,2021-05-05,2021-05-05 04:00:00,258033,0,0
2,0x750b4d06db061fdc59cdf21a3f5a24518eb0f469a255...,0x750b4d06db061fdc59cdf21a3f5a24518eb0f469a255...,"05/05/2021, 05:22:07",1620163327,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0x39336a270b92a64a68925fb8540e7b4adf218e0a,0x39336a270b92a64a68925fb8540e7b4adf218e0a,WBTC,WETH,...,0.010000,0.000000,NaN,258048,349,2021-05-05,2021-05-05 05:00:00,258048,0,0
3,0x0b4a29f4ce37f0c53300209985a4ac78e6d8f2b38b12...,0x0b4a29f4ce37f0c53300209985a4ac78e6d8f2b38b12...,"06/05/2021, 02:16:19",1620238579,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0xe59a4be85f063f1080a0443e6a5fa1dc398a9421,0xe59a4be85f063f1080a0443e6a5fa1dc398a9421,WBTC,WETH,...,1.000000,1714.019835,NaN,258305,223,2021-05-06,2021-05-06 02:00:00,258434,"2,400,506,231,968,713-4,221,963,868,299,692","2,400,506,231,968,713-4,221,963,868,299,692"
4,0x73019fd9817196abc3006265899d970ab18e3078f9d0...,0x73019fd9817196abc3006265899d970ab18e3078f9d0...,"06/05/2021, 02:46:25",1620240385,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xe592427a0aece92de3edee1f18e0157c05861564,0x05ab4c6ab83658e88ff785ca1e78d1519762badb,0x05ab4c6ab83658e88ff785ca1e78d1519762badb,WBTC,WETH,...,0.500000,851.814472,NaN,258434,151,2021-05-06,2021-05-06 02:00:00,258434,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200142,0x9af0035b5112f5e8d50d67483c80fb2224c7930fcd3b...,0x9af0035b5112f5e8d50d67483c80fb2224c7930fcd3b...,"03/01/2024, 09:41:59",1704246119,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,0x00000000d7c185343e6504e428b8f8b5ad6c91b8,0x00000000d7c185343e6504e428b8f8b5ad6c91b8,WBTC,WETH,...,1.305000,3083.413694,NaN,259747,500,2024-01-03,2024-01-03 09:00:00,259747,0,0
200143,0x8027472a11589e1b350cbe2dd8b5be17777ab1e28eff...,0x8027472a11589e1b350cbe2dd8b5be17777ab1e28eff...,"03/01/2024, 11:14:11",1704251651,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0xb64619712b73fb0e6740040af205a0f8134ef8f9,WBTC,WETH,...,21.447268,50815.266478,NaN,259749,29,2024-01-03,2024-01-03 11:00:00,259754,0,0
200144,0xd8aa26bf8fc3674f9acec6c88fbe3c74583a35e191a4...,0xd8aa26bf8fc3674f9acec6c88fbe3c74583a35e191a4...,"03/01/2024, 11:14:23",1704251663,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0xf72837a9fce033d57094f52e4f88f129502fa067,WBTC,WETH,...,18.526282,43894.742678,NaN,259750,34,2024-01-03,2024-01-03 11:00:00,259754,0,0
200145,0x532d5c625b6134e52d8358500a062a648dd833c30052...,0x532d5c625b6134e52d8358500a062a648dd833c30052...,"03/01/2024, 11:14:35",1704251675,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x767c8bb1574bee5d4fe35e27e0003c89d43c5121,0x2d722c96f79d149dd21e9ef36f93fc12906ce9f8,0xb2ce0eb5b5134c734bf1a131257275432f33a4f4,WBTC,WETH,...,39.256197,93007.533053,NaN,259752,37,2024-01-03,2024-01-03 11:00:

In [22]:
df_merged = swaps_merged[['ID_x', 'Transaction ID', 'Current Day', 'Timestamp', 'Timestamp (Unix)', 'Pool_x',
       'Sender', 'Recipient', 'Origin',
       'Token 0', 'Token 1', 'Amount 0', 'Amount 1', 'Amount USD',
       'SqrticeX96', 'Tick_x', 'Log Index', 'Tick Index', 'Fee Growth Outside 0 X128', 'Fee Growth Outside 1 X128', 'Liquidity Gross',
       'Liquidity Net', 'Volume Token 0', 'Volume Token 1', 'Volume USD']]

df_merged.rename(columns={'ID_x': 'ID', 'Tick_x': 'Swap Tick', 'Pool_x': 'Pool'}, inplace=True)


C:\Users\yinki\AppData\Local\Temp\ipykernel_13672\592980226.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged.rename(columns={'ID_x': 'ID', 'Tick_x': 'Swap Tick', 'Pool_x': 'Pool'}, inplace=True)


In [24]:
# Check nan values
print(df_merged.isna().sum())



df_merged['Liquidity Net'] = df_merged['Liquidity Net'].apply(lambda x: int(str(x).replace(',', '')))
df_merged['Liquidity Gross'] = df_merged['Liquidity Gross'].apply(lambda x: int(str(x).replace(',', '')))
df_merged

ID                                0
Transaction ID                    0
Current Day                       0
Timestamp                         0
Timestamp (Unix)                  0
Pool                              0
Sender                            0
Recipient                         0
Origin                            0
Token 0                           0
Token 1                           0
Amount 0                          0
Amount 1                          0
Amount USD                        0
SqrticeX96                   200147
Swap Tick                         0
Log Index                         0
Tick Index                   196844
Fee Growth Outside 0 X128    196844
Fee Growth Outside 1 X128    196844
Liquidity Gross              196844
Liquidity Net                196844
Volume Token 0               196844
Volume Token 1               196844
Volume USD                   196844
dtype: int64


ValueError: invalid literal for int() with base 10: 'nan'

In [64]:
import pandas as pd
from datetime import datetime
# Import necessary functions from your tools.py file
from models.tools import get_liquidity, amounts_relation, get_amounts

# Step 1: Download all the swaps from The Graph
# Assuming df_swaps is your swaps DataFrame
# You need to fetch swaps data from The Graph API or another source
# df_swaps = ...

# Step 2: Download historical data of pool liquidity
# Assuming df_liquidity is your liquidity DataFrame
# You need to fetch liquidity data from a web3 client with archive node access or The Graph API
# df_liquidity = ...

# Step 3: Merge swaps data with liquidity data
# Assuming df_merged is your merged DataFrame
#df_merged = pd.merge(df_swaps, df_liquidity, how='left', left_on='Starting Hour', right_on='Timestamp')

df_merged = swaps_merged.copy()

# Step 4: Define a position to be simulated
initial_capital = 1000000  # Set your initial capital amount ($)
price_at_mint = 0.067304  # Set the price at mint
tick_lower = 0.04177481929059751  # Set tick_lower
tick_upper = 0.07653292116574624  # Set tick_upper
tick_at_mint = df_merged.loc[0, 'Pool Tick']  # Set the tick at mint


liquidity = get_liquidity(tick_at_mint, tick_lower, tick_upper, initial_capital, price_at_mint)

# Step 5: Calculate PNL for the position
fees_accrued = 0
gas_price = 0.0000000001
impermanent_loss = 0
gas_costs_mint = 500000 * gas_price  # Set your gas price

for index, swap in df_merged.iterrows():
    # Assuming you have the necessary data for the swap
    swap_amount_in = swap['Amount In']
    fee_tier = 0.0003  # Assuming constant fee tier for the full hour
    position_liquidity = liquidity  # Assuming constant liquidity for the full hour

    fees_accrued += swap_amount_in * fee_tier * (position_liquidity / swap['Active Liquidity'])
    # Calculate impermanent loss using your formula
    # ...

pnl = fees_accrued - impermanent_loss - gas_costs_mint
age_of_position = (datetime.now() - datetime.strptime('Starting Hour', '%Y-%m-%d %H:%M:%S')).days
apr = pnl / initial_capital * (age_of_position / 365)  # Assuming year time

# Display or use the results
print(f"Fees Accrued: {fees_accrued}, Impermanent Loss: {impermanent_loss}, PNL: {pnl}, APR: {apr}")

TypeError: get_liquidity() missing 2 required positional arguments: 'decimal0' and 'decimal1'

In [75]:
import numpy as np 
import pandas as pd 
from scipy.stats import norm 
 
def black_scholes(S0, X, T, r, sigma): 
    """ 
    Calculate the Black-Scholes option price for a European call option. 
 
    :param S0: Current price of the underlying stock 
    :param X: Strike price of the option 
    :param T: Time to expiration in years 
    :param r: Risk-free interest rate 
    :param sigma: Volatility of the stock's returns 
    :return: Call option price 
    """ 
    d1 = (np.log(S0 / X) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T)) 
    d2 = d1 - sigma * np.sqrt(T) 
    call_price = (S0 * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d2)) 
    return call_price 
 
def calculate_strike_prices(S0, r, T, sigma, min_range=-50, max_range=120, step=10): 
    """ 
    Calculate call option prices for a range of strike prices. 
 
    :param S0: Current price of the underlying stock 
    :param r: Risk-free interest rate 
    :param T: Time to expiration in years 
    :param sigma: Volatility of the stock's returns 
    :param min_range: Minimum percentage range for strike price 
    :param max_range: Maximum percentage range for strike price 
    :param step: Percentage step between strike prices 
    :return: DataFrame with strike prices and corresponding option prices 
    """ 
    strike_prices = [S0 * (1 + i / 100) for i in range(min_range, max_range + 1, step)] 
    option_prices = [black_scholes(S0, strike, T, r, sigma) for strike in strike_prices] 
 
    df = pd.DataFrame({ 
        'Strike Price': strike_prices, 
        'Option Price': option_prices 
    }) 
 
    return df 
 
# Example usage 
S0 = 100  # Close Price
r = 0.05  # Risk-free interest rate 
T = 1     # Time to expiration (length of investment window) D / 365
sigma = 0.2  # forecasted daily volatility of the stock's returns
 
# Calculate option prices for the specified range 
df = calculate_strike_prices(S0, r, T, sigma) 
 
# Save to CSV 
csv_path = "black_scholes_option_prices.csv" 
df.to_csv(csv_path, index=False) 
 
csv_path

'black_scholes_option_prices.csv'